<br>

## random shit

In [ ]:
import sys
#!{sys.executable} -m pip install httpx
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install sweetviz
#!{sys.executable} -m pip install ipyleaflet

<br>

## import libraries

In [ ]:
import pandas as pd
import httpx
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import sweetviz as sv
import seaborn as sns
from ipyleaflet import Map, Marker, basemaps, LayersControl


In [ ]:
#pip install plotly

<br>

## scrape home

In [ ]:
URI = "https://home.dk/umbraco/backoffice/home-api/Search"

params_k = dict(CurrentPageNumber=2,
              SearchResultsPerPage=800,
              q="København kommune",
              BoligKanLejes=True,
              BoligTilSalg=True,
              SearchType=0)

params_f = dict(CurrentPageNumber=2,
              SearchResultsPerPage=800,
              q="Frederiksberg kommune",
              BoligKanLejes=True,
              BoligTilSalg=True,
              SearchType=0)

In [ ]:
scrape_k = httpx.get(url=URI, params=params_k)
scrape_f = httpx.get(url=URI, params=params_f)

In [ ]:
data_k = scrape_k.json()
data_f = scrape_f.json()
#data.keys()

In [ ]:
#pd.DataFrame(data["searchResults"])

<br>

## scrape boliga

In [ ]:
URI = "https://api.boliga.dk/api/v2/sold/search/results"

params_k = dict(searchTab=1,
              propertyType=3,
              municipality=101,
              page=1,
              sort="date-d",
)

params_f = dict(searchTab=1,
              propertyType=3,
              municipality=147,
              page=1,
              sort="date-d",
)

In [ ]:
r_k = httpx.get(url=URI, params=params_k)
r_f = httpx.get(url=URI, params=params_f)

In [ ]:
data_k = r_k.json()
data_f = r_f.json()
pages_k = data_k["meta"]["totalPages"] + 1 # Python starts are zero
pages_f = data_f["meta"]["totalPages"] + 1 # Python starts are zero

In [ ]:
def scrape_to_data(pages, params):
    storage = []
    for page in range(1, pages):

        params["page"] = page
        r = httpx.get(url=URI, params=params, timeout=30)

        storage.extend(r.json()["results"])

        if page % 100 == 0:
            print(f'{page} of pages')
    
    print(f"completed {pages} pages")

    return pd.DataFrame(storage)

df_k = scrape_to_data(pages_k, params_k)
df_f = scrape_to_data(pages_f, params_f)

In [ ]:
df = pd.concat([df_f,df_k])
df

<br>

### Conversion to csv file

In [ ]:
df.to_csv("bolig.csv.gzip", compression="gzip", index=False)

<br>

### Reading the csv file so there is no need for scraping the website boliga again. 
 Last scraped Tuesday the 26.09.2023

In [ ]:
df = pd.read_csv("assets/data/bolig.csv.gzip", compression="gzip")
df

<br>

## Importér prisindex

In [ ]:
pris_index = pd.read_csv("prisindex.csv",sep=";")
pris_index.T.reset_index()
#collect the year 1992-2023 and the right region (københavn+frederiksberg)

<br>

## Data filtration

In [ ]:
alm_salg_df = df[df['saleType'] == 'Alm. Salg']


In [ ]:
alm_salg_df

<br>

### Analysis Tool

In [ ]:
report = sv.analyze(alm_salg_df)
report.show_html()

<br>

## Most Common 

In [ ]:
alm_salg_df[['rooms','price','zipCode','buildYear','sqmPrice']].describe().T

In [ ]:
alm_salg_df[['rooms']].value_counts(normalize=True)
#not include something that is under 0.001%procent
#include from 0.05%<
#the roomes should be from 1-5 

In [ ]:
alm_salg_df['price'].min()


In [ ]:
pd.cut(alm_salg_df['price'],15)

In [ ]:
most_common_room = alm_salg_df['rooms'].value_counts().idxmax()
print('Most common number of roomes:',most_common_room)

In [ ]:
most_common_price = alm_salg_df['price'].value_counts().idxmax()
print('Most common salespice:', most_common_price)

In [ ]:
most_common_zip =  alm_salg_df['zipCode'].value_counts().idxmax()
print('Most common zip-code:',most_common_zip)

In [ ]:
most_common_buildYear = alm_salg_df['buildYear'].value_counts().idxmax()
print('Most common build year:',most_common_buildYear)

In [ ]:
mean_sqmPrice = alm_salg_df['sqmPrice'].mean()
print('Most common square meter price:',mean_sqmPrice)

In [ ]:
import matplotlib.pyplot as plt

<br>

## Histogrammer

<br>

Filtrering af lejligheder der koster over en halv mio

In [ ]:
alm_salg_df[alm_salg_df['price'] < 5e5]

<br>

### Filtrering så vi har salgspriser fra 500.000 til 15 mio

In [ ]:
alm_salg_df[(alm_salg_df['price'] > 5e5) & (alm_salg_df['price'] < 15e6)]['sqmPrice'].hist(bins=100)

In [ ]:
print('First sold appartment date:',df['soldDate'].min() )


In [ ]:
alm_salg_df[(alm_salg_df['zipCode'] < 1650) & (alm_salg_df['sqmPrice'] < 200000)]['sqmPrice'].hist()

<br>

## Scatter Plot af KBH

In [ ]:
cities = alm_salg_df.groupby(['city']).size().reset_index(name='count')
cities

In [ ]:
# Group by 'city' and get the distinct zip codes for each city
city_zip_codes = alm_salg_df.groupby('city')['zipCode'].unique().reset_index(name='zip_codes')

# 'city_zip_codes' now contains a list of distinct zip codes for each city
city_zip_codes

In [ ]:
# Specify the target city
target_city = 'København K'

# Find the row for the target city
city_row = city_zip_codes[city_zip_codes['city'] == target_city]

# Check if the target city exists in the DataFrame
if not city_row.empty:
    zip_codes_for_target_city = city_row['zip_codes'].iloc[0]
    print(f"Zip codes for {target_city}: {', '.join(map(str, zip_codes_for_target_city))}")
else:
    print(f"{target_city} not found in the grouped data")


In [ ]:
# Step 1: Calculate the mean sold price for each city
mean_prices_city = alm_salg_df.groupby('city')['price'].mean().reset_index()

# Step 2: Merge the mean sold price back into the original DataFrame
alm_salg_df_groupbby_city = alm_salg_df.merge(mean_prices_city, on='city', suffixes=('', '_mean'))

# Step 3: Create a heatmap
plt.figure(figsize=(10, 8))
sns.scatterplot(data=alm_salg_df_groupbby_city, x='longitude', y='latitude', hue='price_mean', palette='YlOrRd', s=20)
plt.title('Heatmap of Apartments with Mean Sold Prices by City')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='Mean Sold Price', loc='upper right')
plt.savefig('heatmap_meansold.pdf')
plt.show()


In [178]:
alm_salg_df_zipCode = alm_salg_df.copy

alm_salg_df_zipCode['price'] = np.log(alm_salg_df['price'])

#Step 1: Calculate the mean sold price for each city
mean_prices_zipCode = alm_salg_df_zipCode.groupby('zipCode')['price'].mean().reset_index()

alm_salg_df_zipCode['price'] = alm_salg_df_zipCode['price']**10

# Step 2: Merge the mean sold price back into the original DataFrame
alm_salg_df_groupbby_zipCode = alm_salg_df_zipCode.merge(mean_prices_zipCode, on='zipCode', suffixes=('', '_mean'))

# Step 3: Create a heatmap
plt.figure(figsize=(10, 8))
sns.scatterplot(data=alm_salg_df_groupbby_zipCode, x=('longitude'), y='latitude', hue='price_mean', palette='YlOrRd', s=20)
plt.title('Heatmap of Apartments with Mean Sold Prices by zipCode')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='Mean Sold Price', loc='upper right')
plt.savefig('heatmap_meansold.pdf')
plt.show()


TypeError: 'method' object does not support item assignment

Make this work with the "ipyleaflet: Interactive maps in the Jupyter notebook"


In [ ]:
# Create a map centered at a specific location (you can change the coordinates)
center = (latitude, longitude)
m = Map(center=center, zoom=10, basemap=basemaps.Esri.NatGeoWorldMap)



In [ ]:
for index, row in alm_salg_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    marker = Marker(location=(latitude, longitude), title=row['address'])
    m.add_layer(marker)


In [ ]:
m.add_control(LayersControl())
m

In [ ]:
plt.scatter(df['longitude'], df['latitude'], s= .01, alpha=.5, )

In [ ]:
alm_salg_df[['city']].value_counts(normalize=True)

<br>

### Filtering så man sikre sig at Frb kommune salg er med

In [ ]:
df[df['zipCode']==2000]

<br>

### kigger på de borliger der er slogt til mest for at tjekke om filtrering af outliers af boliger der er slogt til mere end 9 mio

In [ ]:
print('top 10 most expencive appartment\n',alm_salg_df['change'].nlargest(10))